In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pypm.optical.signal as sig
import pyhexagon.processing.spectrums as spc

In [ ]:
# fs = 80e6 / 2**17 / 10 / 6 / 3
sps = 80e6
duration = 5e-3 # duration of synthetic data (sec)
tau = np.arange(0,duration,1/sps) # times (sec)

p_lpsd = {"olap":"default","bmin":1,"Lmin":0,
           "Jdes":2000,"Kdes":100,"order":0,
           "win":np.kaiser,"psll":250}

# : optical parameters
fhet = 10e6 # carrier-carrier heterodyne beatnote frequency (Hz)
Plo = 1     # local beam power (W)
Ps = 1      # signal (= Received) beam power (W)
gamma = 1   # heterodine efficiency
msb = np.sqrt(2*0.15) # modulation index of clock sidebands (rad)
fsb = 1e6   # sideband beatnote frequency offset from a carrier beatnote (Hz)
eta = 0.7   # photoreceiver responsivity (A/W)
Rf = 30.1e3 # photoreceiver TIA gain (V/A)

s = sig.Signal(Plo=Plo, Ps=Ps, gamma=gamma, fhet=fhet, msb=msb, fsb=fsb, eta=eta, Rf=Rf)

s.define_noises(seed=0, carrier_asd=60, sps=sps)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.plot(s.sequence(tau=tau)[:300])
ax.set_xlabel("Sample")
ax.set_ylabel("Signal")

In [ ]:
pn_cc = s.bn_cc.phase_noise(size=tau.size)
pn_usb = s.bn_sbu.phase_noise(size=tau.size)
pn_lsb = s.bn_sbl.phase_noise(size=tau.size)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)

ax.plot(tau, pn_usb, label='US beatnote phase(t)')
ax.plot(tau, pn_lsb, label='LS beatnote phase(t)', linestyle=':')
ax.plot(tau, pn_cc,  label='CC beatnote phase(t)')
ax.set_xlabel("Time (s)")
ax.set_ylabel("Signal")
ax.legend()

In [ ]:
f1, asd1, as1 = spc.spectra(pn_cc, fs=sps, p_lpsd='default', use_lasd=True)
f2, asd2, as2 = spc.spectra(pn_usb, fs=sps, p_lpsd='default', use_lasd=True)
f3, asd3, as3 = spc.spectra(pn_lsb, fs=sps, p_lpsd='default', use_lasd=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)

ax.loglog(f1, asd1, label='CC beatnote phase(f)')
ax.loglog(f2, asd2, label='US beatnote phase(f)')
ax.loglog(f3, asd3,  label='LS beatnote phase(f)', linestyle='--')
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_ylabel(r"Amplitude spectral dentisiy ${\rm rad}/\sqrt{\rm Hz}$")
ax.legend()

In [ ]:
p_lpsd = {"olap":"default","bmin":1,"Lmin":100,
           "Jdes":10000,"Kdes":100,"order":0,
           "win":np.kaiser,"psll":250}

f4, asd4, _ = s.spectrum(tau, p_lpsd, use=['cc','sbu','sbl'], dBm=False, use_lpsd=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)

ax.loglog(f4, asd4)
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_ylabel(r"Amplitude spectral dentisiy ${\rm A}/\sqrt{\rm Hz}$")